For this project, I need to build a model that predicts the volume of reserves in new wells, pick the oil wells with the highest profits, and pick the region with the highest total profit. First thing I'll do is import all the necessary libraries to complete these tasks.

In [24]:
import pandas as pd
import numpy as np
from scipy import stats as st
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error

Next thing I'll do is load the data sets.

In [25]:
geodata0 = pd.read_csv('/datasets/geo_data_0.csv')
geodata1 = pd.read_csv('/datasets/geo_data_1.csv')
geodata2 = pd.read_csv('/datasets/geo_data_2.csv')

With the data sets loaded, I'll now start the initial look into them.

In [26]:
geodata0.info()
print('\n')
print(geodata0.isna().sum())
display(geodata0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [27]:
geodata1.info()
print('\n')
print(geodata1.isna().sum())
display(geodata1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


In [28]:
geodata2.info()
print('\n')
print(geodata2.isna().sum())
display(geodata2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


I'll check for any obvious duplicates next.

In [29]:
duplicates0 = geodata0.duplicated()
duplicates1 = geodata1.duplicated()
duplicates2 = geodata2.duplicated()
print(duplicates0.sum())
print(duplicates1.sum())
print(duplicates2.sum())

0
0
0


With no missing values and no duplicates, I'm ready to split the data into features and target variable.

In [30]:
features0 = geodata0.drop(columns=['id', 'product'])
target0 = geodata0['product']

In [31]:
features1 = geodata1.drop(columns=['id', 'product'])
target1 = geodata1['product']

In [32]:
features2 = geodata2.drop(columns=['id', 'product'])
target2 = geodata2['product']

I'll split the data into training and validation sets for each dataset.

In [33]:
features0_train, features0_valid, target0_train, target0_valid = train_test_split(
    features0, target0, test_size=0.25, random_state=12345)

In [34]:
features1_train, features1_valid, target1_train, target1_valid = train_test_split(
    features1, target1, test_size=0.25, random_state=12345)

In [35]:
features2_train, features2_valid, target2_train, target2_valid = train_test_split(
    features2, target2, test_size=0.25, random_state=12345)

Now I'll train a linear regression model on each training set and find the average volume of predicted reserves, and calculate the RMSE for the region.

In [36]:
model0 = LinearRegression()
model0.fit(features0_train, target0_train)
predictions0_valid = model0.predict(features0_valid)

result0 = mean_squared_error(target0_valid, predictions0_valid)**0.5
average_volume0 = predictions0_valid.mean()

print("Average volume of predicted reserves in 'Geodata0':", average_volume0)
print("RMSE of 'Geodata0':", result0)

Average volume of predicted reserves in 'Geodata0': 92.59256778438035
RMSE of 'Geodata0': 37.5794217150813


In [37]:
model1 = LinearRegression()
model1.fit(features1_train, target1_train)
predictions1_valid = model1.predict(features1_valid)

result1 = mean_squared_error(target1_valid, predictions1_valid)**0.5
average_volume1 = predictions1_valid.mean()

print("Average volume of predicted reserves in 'Geodata1':", average_volume1)
print("RMSE of 'Geodata1':", result1)

Average volume of predicted reserves in 'Geodata1': 68.728546895446
RMSE of 'Geodata1': 0.893099286775617


In [38]:
model2 = LinearRegression()
model2.fit(features2_train, target2_train)
predictions2_valid = model2.predict(features2_valid)

result2 = mean_squared_error(target2_valid, predictions2_valid)**0.5
average_volume2 = predictions2_valid.mean()

print("Average volume of predicted reserves in 'Geodata2':", average_volume2)
print("RMSE of 'Geodata2':", result2)

Average volume of predicted reserves in 'Geodata2': 94.96504596800489
RMSE of 'Geodata2': 40.02970873393434


Based on the results above, it appears that 'Geodata1' has the closest predictions to the actual values in the region.

Now I'll move on to preparing the profit calculations of each region. I'll do this by storing the key values in different variables first.

In [39]:
revenue_per_barrel = 4500
budget_for_dev = 100000000
number_of_wells = 200

Now I'll find the cost of the barrels per region.

In [40]:
cost_per_barrel0 = budget_for_dev / (number_of_wells * average_volume0)
cost_per_barrel1 = budget_for_dev / (number_of_wells * average_volume1)
cost_per_barrel2 = budget_for_dev / (number_of_wells * average_volume2)

Next I'll find the profit per barrel per region.

In [41]:
profit_per_barrel0 = revenue_per_barrel - cost_per_barrel0
profit_per_barrel1 = revenue_per_barrel - cost_per_barrel1
profit_per_barrel2 = revenue_per_barrel - cost_per_barrel2

Now calculate the cost per barrel, the profit per barrel, and the total profit for each region.

In [42]:
total_profit0 = average_volume0 * profit_per_barrel0
total_profit1 = average_volume1 * profit_per_barrel1
total_profit2 = average_volume2 * profit_per_barrel2

In [43]:
print("Cost per barrel for 'Geodata0':", cost_per_barrel0)
print("Cost per barrel for 'Geodata1':", cost_per_barrel1)
print("Cost per barrel for 'Geodata2':", cost_per_barrel2)
print("Profit per barrel for 'Geodata0':", profit_per_barrel0)
print("Profit per barrel for 'Geodata1':", profit_per_barrel1)
print("Profit per barrel for 'Geodata2':", profit_per_barrel2)
print("Total profit for 'Geodata0':", total_profit0)
print("Total profit for 'Geodata1':", total_profit1)
print("Total profit for 'Geodata2':", total_profit2)

Cost per barrel for 'Geodata0': 5400.001446815326
Cost per barrel for 'Geodata1': 7274.99740043435
Cost per barrel for 'Geodata2': 5265.095118981539
Profit per barrel for 'Geodata0': -900.0014468153258
Profit per barrel for 'Geodata1': -2774.9974004343503
Profit per barrel for 'Geodata2': -765.0951189815387
Total profit for 'Geodata0': -83333.44497028844
Total profit for 'Geodata1': -190721.53897049298
Total profit for 'Geodata2': -72657.293143978


With this data, it shows that each region would result in a loss of profit. The region I have labeled as "Geodata2" would have the least amount of loss, followed by "Geodata0" and then "Geodata1" having the highest loss.

Now I will create a function to calculate profit from a set of selected oil wells and model predictions.

In [44]:
def calc_profit(wells, predictions, target_volumes, revenue_per_barrel, budget_for_dev):
    sorted_indices = predictions.argsort()[::-1]
    selected_indices = sorted_indices[:wells]
    
    target_volumes_reset_index = target_volumes.reset_index(drop=True)
    
    total_volume = target_volumes_reset_index[selected_indices].sum()

    cost_per_barrel = budget_for_dev / wells
    profit = revenue_per_barrel * total_volume - cost_per_barrel * wells

    return profit


I'll find the profit for each region and go from there.

In [45]:
wells = 200

profit0 = calc_profit(wells, predictions0_valid, target0_valid, revenue_per_barrel, budget_for_dev)
print("Total profit for 'Geodata0' with selected wells:", profit0)

profit1 = calc_profit(wells, predictions1_valid, target1_valid, revenue_per_barrel, budget_for_dev)
print("Total profit for 'Geodata1' with selected wells:", profit1)

profit2 = calc_profit(wells, predictions2_valid, target2_valid, revenue_per_barrel, budget_for_dev)
print("Total profit for 'Geodata2' with selected wells:", profit2)

Total profit for 'Geodata0' with selected wells: 33208260.43139851
Total profit for 'Geodata1' with selected wells: 24150866.966815114
Total profit for 'Geodata2' with selected wells: 27103499.635998324


From the function and with determining the profit for 200 wells, the best option for profit would be the region labeled "Geodata0."

Now I'll create a function for calculating the risks and profit for each region using the bootstrap technique.

In [47]:
def bootstrap_profit(predictions, target_volumes, revenue_per_barrel, budget_for_dev, num_samples=1000, sample_points=500):
    profits = []

    for i in range(num_samples):
        sample_indices = np.random.choice(len(predictions), sample_points, replace=True)
        sample_predictions = predictions[sample_indices]
        sample_target_volumes = target_volumes.reset_index(drop=True)[sample_indices]

        profit = calc_profit(wells, sample_predictions, sample_target_volumes, revenue_per_barrel, budget_for_dev)
        profits.append(profit)

    avg_profit = np.mean(profits)
    confidence_interval = np.percentile(profits, [2.5, 97.5])
    risk_of_losses = np.mean(np.array(profits) < 0) * 100

    return avg_profit, confidence_interval, risk_of_losses

avg_profit0, confidence_interval0, risk_of_losses0 = bootstrap_profit(predictions0_valid, target0_valid, revenue_per_barrel, budget_for_dev)
avg_profit1, confidence_interval1, risk_of_losses1 = bootstrap_profit(predictions1_valid, target1_valid, revenue_per_barrel, budget_for_dev)
avg_profit2, confidence_interval2, risk_of_losses2 = bootstrap_profit(predictions2_valid, target2_valid, revenue_per_barrel, budget_for_dev)


In [48]:
print("Region 'Geodata0':")
print("Average Profit:", avg_profit0)
print("95% Confidence Interval:", confidence_interval0)
print("Risk of Losses:", risk_of_losses0)

print("\nRegion 'Geodata1':")
print("Average Profit:", avg_profit1)
print("95% Confidence Interval:", confidence_interval1)
print("Risk of Losses:", risk_of_losses1)

print("\nRegion 'Geodata2':")
print("Average Profit:", avg_profit2)
print("95% Confidence Interval:", confidence_interval2)
print("Risk of Losses:", risk_of_losses2)

Region 'Geodata0':
Average Profit: 3864862.1572553026
95% Confidence Interval: [-1309870.80262279  9242956.82459124]
Risk of Losses: 7.3999999999999995

Region 'Geodata1':
Average Profit: 4514331.7475810945
95% Confidence Interval: [ 381822.72698609 8701221.46030057]
Risk of Losses: 1.7000000000000002

Region 'Geodata2':
Average Profit: 3920331.862260799
95% Confidence Interval: [-1365647.67588584  9337404.13315174]
Risk of Losses: 6.2


To maximize profit, my suggestion would be to develop wells in the region "Geodata1". "Geodata1" is expected to generate the most profit among the three regions, considering the predicted reserves, the revenue per barrel, and the budget constraints, while also having the lowest risk of loss out of the three regions, coming in at only 1.7. If there are limit resources and the goal is to choose the most profitable region for oil well development, that is the region to persue.